In [1]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'


In [50]:
labels = [{'name':'Gao lut', 'id':1},
          {'name':'Gao Huyet Rong', 'id':2},
          {'name':'Gao Lai Sua', 'id':3},
          {'name':'Gao Long Lai', 'id':4},
          {'name':'Gao Lut Bac', 'id':5},
          {'name':'Gao Lut Dien Bien', 'id':6}
          , {'name':'Gao Lut Do', 'id':7},
          {'name':'Gao Lut Trang', 'id':8},
          {'name':'Gao Nep Cam', 'id':9},
          {'name':'Gao Nep Sap', 'id':10},
          {'name':'Gao Ngoc Troi', 'id':11},
          {'name':'Gao Tam', 'id':12},
          {'name':'Gao Tam Thom', 'id':13},
          {'name':'Gao Thom Go Cong', 'id':14},
          {'name':'Gao Thom My', 'id':15}]

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [32]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

Cloning into 'models'...
Updating files:  87% (2681/3075)
Updating files:  88% (2706/3075)
Updating files:  89% (2737/3075)
Updating files:  90% (2768/3075)
Updating files:  91% (2799/3075)
Updating files:  92% (2829/3075)
Updating files:  93% (2860/3075)
Updating files:  94% (2891/3075)
Updating files:  95% (2922/3075)
Updating files:  96% (2952/3075)
Updating files:  97% (2983/3075)
Updating files:  98% (3014/3075)
Updating files:  99% (3045/3075)
Updating files: 100% (3075/3075)
Updating files: 100% (3075/3075), done.


In [3]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [18]:
import tensorflow.compat.v1 as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [31]:
config =config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [32]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 4e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.01
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.997
         scale: true
         epsilon: 0.001
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matcher {
     argmax_matcher {
       matched_threshold: 0.5
   

In [33]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.gfile.GFile(CONFIG_PATH, "r") as f:
   proto_str = f.read()
   text_format.Merge(proto_str, pipeline_config)

In [35]:
pipeline_config.model.ssd.num_classes = 15
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [37]:
pipeline_config

model {
  ssd {
    num_classes: 15
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 4e-05
          }
        }
        initializer {
          random_normal_initializer {
            mean: 0.0
            stddev: 0.01
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.997
          scale: true
          epsilon: 0.001
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
        additional_layer_depth: 128
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      ar

In [38]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text) 